# Глава 10. 
## Снижение размерности с помощью отбора признаков
> <b>10.1 Пороговая обработка дисперсии числовых признаков

In [1]:
# Загрузить библиотеки
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

# Импортировать немного данных для экспериментирования
iris = datasets.load_iris()

# Создать признаки и цель
features = iris.data
target = iris.target

# Создать обработчик порога
thresholder = VarianceThreshold(threshold=.5)

# Создать матрицу высокодисперсионных признаков
features_high_variance = thresholder.fit_transform(features)

# Взглянуть на матрицу высокодисперсионных признаков
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

In [2]:
# Взглянуть на дисперсии
thresholder.fit(features).variances_

array([0.68112222, 0.18675067, 3.09242489, 0.57853156])

In [3]:
# Загрузить библиотеку
from sklearn.preprocessing import StandardScaler

# Стандартизировать матрицу признаков
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

# Вычислить дисперсию каждого признака
selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

> <b>10.2 Пороговая обработка дисперсии бинарных признаков

In [4]:
# Загрузить библиотеку
from sklearn.feature_selection import VarianceThreshold

# Создать матрицу признаков, где:
# признак 0: 80% класс 0
# признак 1: 80% класс 1
# признак 2: 60% класс 0, 40% класс 1
features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# Выполнить пороговую обработку по дисперсии
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

> <b>10.3 Обработка выскокореллированных признаков

In [5]:
# Загрузить библиотеки
import pandas as pd
import numpy as np

# Создать матрицу признаков с высококоррелированными признаками
features = np.array([[1, 1, 1],
                     [2, 2, 0],
                     [3, 3, 1],
                     [4, 4, 0],
                     [5, 5, 1],
                     [6, 6, 0],
                     [7, 7, 1],
                     [8, 7, 0],
                     [9, 7, 1]])

# Конвертировать матрицу признаков во фрейм данных
dataframe = pd.DataFrame(features)

# Создать корреляционную матрицу
corr_matrix = dataframe.corr().abs()

# Выбрать верхний треугольник корреляционной матрицы
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
k=1).astype(np.bool))

# Найти индекс столбцов признаков с корреляцией больше 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Исключить признаки
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3)

,0,2
0,1,1
1,2,0
2,3,1


In [6]:
# Корреляционная матрица
dataframe.corr()

,0,1,2
0,1.000000,0.976103,0.000000
1,0.976103,1.000000,-0.034503
2,0.000000,-0.034503,1.000000


In [7]:
# Верхний треугольник корреляционной матрицы
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


> <b>10.4 Удаление нерелевантных признаков для классификации

In [8]:
# Загрузить библиотеки
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

# Загрузить данные
iris = load_iris()
features = iris.data
target = iris.target

# Конвертировать в категориальные данные путем
# преобразования в данные в целые числа
features = features.astype(int)

# Отобрать два признака с наивысшими значениями  
# статистического показателя хи-квадрат
chi2_selector = SelectKBest(chi2, k=2)
features_kbest = chi2_selector.fit_transform(features, target)

# Показать результаты
print("Исходное количество признаков:", features.shape[1])
print("Сокращенное количество признаков:", features_kbest.shape[1])

Исходное количество признаков: 4
Сокращенное количество признаков: 2


In [9]:
# Отобрать два признака с наивысшими значениями
# статистического показателя F
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

# Показать результаты
print("Исходное количество признаков:", features.shape[1])
print("Сокращенное количество признаков:", features_kbest.shape[1])

Исходное количество признаков: 4
Сокращенное количество признаков: 2


In [10]:
# Загрузить библиотеку
from sklearn.feature_selection import SelectPercentile

# Отобрать верхние 75% признаков с наивысшими значениями
# статистического показателя F
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

# Показать результаты
print("Исходное количество признаков:", features.shape[1])
print("Сокращенное количество признаков:", features_kbest.shape[1])

Исходное количество признаков: 4
Сокращенное количество признаков: 3


> <b>10.5 Рекурсивное устранение признаков

In [32]:
# Загрузить библиотеки
import warnings
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# Подавить раздражающее, но безвредное предупреждение
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")

# Сгенерировать матрицу признаков, вектор целей и истинные коэффициенты
features, target = make_regression(n_samples = 10000,
                                   n_features = 100,
                                   n_informative = 2,
                                   random_state = 1)

# Создать объект линейной регрессии
ols = linear_model.LinearRegression()

# Рекурсивно устранить признаки
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[-0.76165969,  0.00850799, -1.72570086, ..., -1.15861018,
        -0.29639545, -0.88199355],
       [-0.46550514, -1.07500204, -1.65067126, ...,  0.42556217,
         0.53734944, -0.57765351],
       [ 1.36836958,  1.37940721,  0.24628915, ...,  0.70017292,
        -0.32395153, -0.17692642],
       ...,
       [ 0.39023196, -0.80331656,  0.71806912, ...,  0.44188555,
         0.13996708,  0.98775903],
       [ 0.44825266,  0.39508844,  0.01838076, ..., -0.74637847,
         0.11077605,  1.00252214],
       [ 1.15616404, -0.55383035, -0.20621076, ..., -0.1457644 ,
         0.19528325, -0.04260289]])

In [33]:
# Количество самых лучших признаков
rfecv.n_features_

9

In [34]:
# Какие категории самые лучшие
rfecv.support_

array([False,  True, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False])

In [35]:
# Ранжировать признаки от самого лучшего (1) до самого плохого
rfecv.ranking_

array([48,  1, 51, 57, 13,  1, 14, 85, 56, 65,  1,  6, 81, 41, 82,  2, 55,
       78, 20,  8, 92,  1, 90, 23, 59, 80, 24, 88, 39,  1, 42, 11, 34, 47,
       71, 69, 27, 72, 76,  1, 31, 91, 54, 61, 74, 75, 64, 89,  5, 19, 43,
       25,  3, 66, 52, 22, 26, 60, 12, 30, 87,  7, 37, 35, 15, 38, 18, 10,
       58,  1,  9, 28, 40, 33, 79, 68, 46, 16,  4, 84, 36,  1, 44, 21, 50,
       83, 63, 53, 86, 70, 62,  1, 77, 32, 67, 45, 29, 73, 49, 17])